# Exporting model from PyTorch to ONNX

In this tutorial, we describe how to use ONNX to convert a model defined
in PyTorch into the ONNX format.

ONNX exporter is part of the [PyTorch repository](http://pytorch.org/docs/master/onnx.html).

For working with this tutorial, you will need to install [onnx](https://github.com/onnx/onnx). You can get binary builds of onnx with
``conda install -c conda-forge onnx``.

``NOTE``: ONNX is under active development so for the best support consider building PyTorch master branch which can be installed by following
[the instructions here](https://github.com/pytorch/pytorch#from-source)

## Invoking exporter

Pretty much it's a matter of replacing `my_model(input)` with `torch.onnx.export(my_model, input, "my_model.onnx")` in your script.

### Limitations

***The ONNX exporter is a trace-based exporter, which means that it operates by executing your model once, and exporting the operators which were actually run during this run. This means that if your model is dynamic, e.g., changes behavior depending on input data, the export won’t be accurate.***

Similarly, a trace is might be valid only for a specific input size (which is one reason why we require explicit inputs on tracing). Most of the operators export size-agnostic versions and should work on different batch sizes or input sizes. We recommend examining the model trace and making sure the traced operators look reasonable.

In [1]:
import torch.onnx
help(torch.onnx.export)




Help on function export in module torch.onnx:

export(model, args, f, export_params=True, verbose=False, training=<TrainingMode.EVAL: 0>, input_names=None, output_names=None, aten=False, export_raw_ir=False, operator_export_type=None, opset_version=None, _retain_param_name=True, do_constant_folding=True, example_outputs=None, strip_doc_string=True, dynamic_axes=None, keep_initializers_as_inputs=None, custom_opsets=None, enable_onnx_checker=True, use_external_data_format=False)
    Export a model into ONNX format.  This exporter runs your model
    once in order to get a trace of its execution to be exported;
    at the moment, it supports a limited set of dynamic models (e.g., RNNs.)
    
    Arguments:
        model (torch.nn.Module): the model to be exported.
        args (tuple of arguments or torch.Tensor): the inputs to
            the model, e.g., such that ``model(*args)`` is a valid
            invocation of the model.  Any non-Tensor arguments will
            be hard-coded in

## Trying it out on MobileNet

If you already have your model built, it's just a few lines:

In [2]:
import torch.onnx
import torchvision

# Standard ImageNet input - 3 channels, 224x224,
# values don't matter as we care about network structure.
# But they can also be real inputs.
dummy_input = torch.randn(1, 3, 224, 224)
# Obtain your model, it can be also constructed in your script explicitly
model = torchvision.models.mobilenet_v2(pretrained=True)
# Invoke export
input_names = [ "input_images" ]
output_names = ["output"]
torch.onnx.export(model, dummy_input, "mobileNetV2.onnx",verbose=True,input_names=input_names, output_names=output_names)

      %619 : Float(960:160, 160:1, 1:1, 1:1, requires_grad=0, device=cpu),
      %620 : Float(960:1, requires_grad=0, device=cpu),
      %622 : Float(960:9, 1:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %623 : Float(960:1, requires_grad=0, device=cpu),
      %625 : Float(320:960, 960:1, 1:1, 1:1, requires_grad=0, device=cpu),
      %626 : Float(320:1, requires_grad=0, device=cpu),
      %628 : Float(1280:320, 320:1, 1:1, 1:1, requires_grad=0, device=cpu),
      %629 : Float(1280:1, requires_grad=0, device=cpu),
      %630 : Long(1:1, requires_grad=0, device=cpu)):
  %474 : Float(1:401408, 32:12544, 112:112, 112:1, requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[1, 1, 1, 1], strides=[2, 2]](%input_images, %475, %476)
  %317 : Float(1:401408, 32:12544, 112:112, 112:1, requires_grad=1, device=cpu) = onnx::Clip[max=6., min=0.](%474) # /home/altex/torch-venv/lib/python3.8/site-packages/torch/nn/functional.py:1186:0
  %477 : Float(1:401408

**That's it!**
The resulting mobileNetV2.onnx is a binary protobuf file which contains both the network structure and parameters of the model you exported

## Inspecting model

You can also use ONNX tooling to check the validity of the resulting model or inspect the details

In [ ]:
import onnx

# Load the ONNX model
model = onnx.load("mobileNetV2.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

Notice that all parameters are listed as graph's inputs but they also have stored values initialized in `model.graph.initializers`.

You can also run the exported model with ONNX Runtime, you will need to install ONNX Runtime: please [follow these instructions](https://github.com/microsoft/onnxruntime#installation).

Once these are installed, you can use the backend for ONNX Runtime:

In [ ]:
import onnxruntime as ort

In [ ]:
ort_session = ort.InferenceSession('mobileNetV2.onnx')

In [ ]:
import numpy as np
outputs = ort_session.run(None, {'input_images': np.random.randn(1, 3, 224, 224).astype(np.float32)})

In [ ]:
outputs[0].shape

# Tracing vs Scripting
The ONNX exporter can be both `trace-based` and `script-based` exporter.

`trace-based` means that it operates by executing your model once, and exporting the operators which were actually run during this run. This means that if your model is dynamic, e.g., changes behavior depending on input data, the export won’t be accurate. Similarly, a trace is likely to be valid only for a specific input size (which is one reason why we require explicit inputs on tracing.) We recommend examining the model trace and making sure the traced operators look reasonable. If your model contains control flows like for loops and if conditions, **trace-based exporter will unroll the loops and if conditions, exporting a static graph that is exactly the same as this run**. If you want to export your model with dynamic control flows, you will need to use the script-based exporter.

script-based means that the model you are trying to export is a ScriptModule. ScriptModule is the core data structure in TorchScript, and TorchScript is a subset of Python language, that creates serializable and optimizable models from PyTorch code.

We allow mixing tracing and scripting. You can compose tracing and scripting to suit the particular requirements of a part of a model. Checkout this example:

In [152]:
import torch  

class LoopModel(torch.nn.Module):
    def __init__(self,n_loop):
        super(LoopModel, self).__init__()
        self.n_loop = n_loop
    def forward(self, x):
        out = x + 1
        for i in range(self.n_loop):
            out += out * i
        return out  

loop = LoopModel(3) 
x = torch.randn(1)
print(x)
out = loop(x)
print(out) 

tensor([-1.2333])
tensor([-1.3999])


In [153]:
traced_module = torch.jit.trace(loop,x)

In [154]:
print(x)
traced_module(x)

tensor([-1.2333])


tensor([-1.3999])

In [155]:
traced_module.graph

graph(%self : __torch__.___torch_mangle_12.LoopModel,
      %x : Float(1:1, requires_grad=0, device=cpu)):
  %3 : Long(requires_grad=0, device=cpu) = prim::Constant[value={1}]() # <ipython-input-152-ea50f94f04b6>:8:0
  %4 : int = prim::Constant[value=1]() # <ipython-input-152-ea50f94f04b6>:8:0
  %out.1 : Float(1:1, requires_grad=0, device=cpu) = aten::add(%x, %3, %4) # <ipython-input-152-ea50f94f04b6>:8:0
  %6 : Long(requires_grad=0, device=cpu) = prim::Constant[value={0}]() # <ipython-input-152-ea50f94f04b6>:10:0
  %7 : Float(1:1, requires_grad=0, device=cpu) = aten::mul(%out.1, %6) # <ipython-input-152-ea50f94f04b6>:10:0
  %8 : int = prim::Constant[value=1]() # <ipython-input-152-ea50f94f04b6>:10:0
  %out.2 : Float(1:1, requires_grad=0, device=cpu) = aten::add_(%out.1, %7, %8) # <ipython-input-152-ea50f94f04b6>:10:0
  %10 : Long(requires_grad=0, device=cpu) = prim::Constant[value={1}]() # <ipython-input-152-ea50f94f04b6>:10:0
  %11 : Float(1:1, requires_grad=0, device=cpu) = aten::mu

In [156]:
traced_module.code

'def forward(self,\n    x: Tensor) -> Tensor:\n  out = torch.add(x, CONSTANTS.c0, alpha=1)\n  out0 = torch.add_(out, torch.mul(out, CONSTANTS.c1), alpha=1)\n  out1 = torch.add_(out0, torch.mul(out0, CONSTANTS.c0), alpha=1)\n  _0 = torch.add_(out1, torch.mul(out1, CONSTANTS.c2), alpha=1)\n  return _0\n'

In [157]:
# With trace-based exporter, we get the result ONNX graph which unrolls the for loop:
torch.onnx.export(LoopModel(3),(x),'loop.onnx', verbose=True)

graph(%0 : Float(1:1, requires_grad=0, device=cpu)):
  %1 : Float(requires_grad=0, device=cpu) = onnx::Constant[value={1}]()
  %2 : Float(1:1, requires_grad=0, device=cpu) = onnx::Add(%0, %1)
  %3 : Float(requires_grad=0, device=cpu) = onnx::Constant[value={0}]()
  %4 : Float(1:1, requires_grad=0, device=cpu) = onnx::Mul(%2, %3)
  %5 : Float(1:1, requires_grad=0, device=cpu) = onnx::Add(%2, %4)
  %6 : Float(requires_grad=0, device=cpu) = onnx::Constant[value={1}]()
  %7 : Float(1:1, requires_grad=0, device=cpu) = onnx::Mul(%5, %6)
  %8 : Float(1:1, requires_grad=0, device=cpu) = onnx::Add(%5, %7)
  %9 : Float(requires_grad=0, device=cpu) = onnx::Constant[value={2}]()
  %10 : Float(1:1, requires_grad=0, device=cpu) = onnx::Mul(%8, %9)
  %11 : Float(1:1, requires_grad=0, device=cpu) = onnx::Add(%8, %10)
  return (%11)



To utilize script-based exporter for capturing the dynamic loop, we can write the loop in script, and call it from the regular nn.Module:

In [158]:
@torch.jit.script
def loop_fn(out,n_loop):
    for i in range(int(n_loop)):
        out += out * i
    return out 
class LoopModel(torch.nn.Module):
    def __init__(self):
        super(LoopModel, self).__init__()
     
    def forward(self, x, loop_count):
        out = x + 1
        out = loop_fn(out, loop_count)
        return out  

In [163]:
loop_count = torch.tensor(5, dtype=torch.long)
torch.onnx.export(LoopModel(),(x,loop_count),'loop.onnx', verbose=True, input_names=['inputs','loop_count'], output_names=['output'])

graph(%inputs : Float(1:1, requires_grad=0, device=cpu),
      %loop_count : Long(requires_grad=0, device=cpu),
      %14 : Bool(requires_grad=0, device=cpu)):
  %2 : Float(requires_grad=0, device=cpu) = onnx::Constant[value={1}]()
  %3 : Float(1:1, requires_grad=0, device=cpu) = onnx::Add(%inputs, %2)
  %4 : Long(requires_grad=0, device=cpu) = onnx::Constant[value={1}]()
  %output : Float(1:1, requires_grad=0, device=cpu) = onnx::Loop(%loop_count, %14, %3) # <ipython-input-158-eede94731347>:3:4
    block0(%i.1 : Long(device=cpu), %cond : bool, %out.7 : Float(1:1, requires_grad=0, device=cpu)):
      %10 : Float(device=cpu) = onnx::Cast[to=1](%i.1)
      %11 : FloatTensor = onnx::Mul(%out.7, %10) # <ipython-input-158-eede94731347>:4:15
      %12 : FloatTensor = onnx::Add(%out.7, %11)
      %13 : bool = onnx::Cast[to=9](%4)
      -> (%13, %12)
  return (%output)



In [164]:
sess = ort.InferenceSession('loop.onnx')

In [171]:
sess.run(None, {'inputs':np.array([1.6]).astype(np.float32), 'loop_count':np.array([2])})

[array([5.2], dtype=float32)]

## Using dictionaries to handle Named Arguments as model inputs
There are two ways to handle models which consist of named parameters or keyword arguments as inputs:

* The first method is to pass all the inputs in the same order as required by the model and pass None values for the keyword arguments that do not require a value to be passed

* The second and more intuitive method is to represent the keyword arguments as key-value pairs where the key represents the name of the argument in the model signature and the value represents the value of the argument to be passed


In [34]:
import torch.nn as nn 

## its not working ............

class Model(nn.Module):
    def forward(self, x, y=None, z=None):
        if y is not None:
            return x*y 
        else:
            return x*z 


x = torch.randn(1)
y = torch.randn(1)
z = torch.randn(1)
torch.onnx.export(Model(),(x,{'y':None,'z':z}),'model.onnx', output_names=['out'],verbose=True)


RuntimeError: Only tuples, lists and Variables supported as JIT inputs/outputs. Dictionaries and strings are also accepted but their usage is not recommended. But got unsupported type NoneType

## Indexing
Tensor indexing in PyTorch is very flexible and complicated. There are two categories of indexing. Both are largely supported in exporting today. If you are experiencing issues exporting indexing that belongs to the supported patterns below, please double check that you are exporting with the latest opset (opset_version=12).



## Getter
This type of indexing occurs on the RHS. Export is supported for ONNX opset version >= 9. E.g.:

In [35]:
data = torch.randn(3, 4)
index = torch.tensor([1, 2])

# RHS indexing is supported in ONNX opset >= 11.
class RHSIndexing(torch.nn.Module):
    def forward(self, data, index):
        return data[index]

out = RHSIndexing()(data, index)

torch.onnx.export(RHSIndexing(), (data, index), 'indexing.onnx', opset_version=9)

# onnxruntime
import onnxruntime
sess = onnxruntime.InferenceSession('indexing.onnx')
out_ort = sess.run(None, {
    sess.get_inputs()[0].name: data.numpy(),
    sess.get_inputs()[1].name: index.numpy(),
})

assert torch.all(torch.eq(out, torch.tensor(out_ort)))

## Below is the list of supported patterns for RHS indexing.
```
# Scalar indices
data[0, 1]

# Slice indices
data[:3]

# Tensor indices
data[torch.tensor([[1, 2], [2, 3]])]
data[torch.tensor([2, 3]), torch.tensor([1, 2])]
data[torch.tensor([[1, 2], [2, 3]]), torch.tensor([2, 3])]
data[torch.tensor([2, 3]), :, torch.tensor([1, 2])]

# Ellipsis
# Not supported in scripting
# i.e. torch.jit.script(model) will fail if model contains this pattern.
# Export is supported under tracing
# i.e. torch.onnx.export(model)
data[...]

# The combination of above
data[2, ..., torch.tensor([2, 1, 3]), 2:4, torch.tensor([[1], [2]])]

# Boolean mask (supported for ONNX opset version >= 11)
data[data != 1]
```

## And below is the list of unsupported patterns for RHS indexing.
```
# Tensor indices that includes negative values.
data[torch.tensor([[1, 2], [2, -3]]), torch.tensor([-2, 3])]
```

## Setter
In code, this type of indexing occurs on the LHS. Export is supported for ONNX opset version >= 11. E.g.:

In [37]:
data = torch.zeros(3, 4)
new_data = torch.arange(4).to(torch.float32)

# LHS indexing is supported in ONNX opset >= 11.
class LHSIndexing(torch.nn.Module):
    def forward(self, data, new_data):
        data[1] = new_data
        return data

out = LHSIndexing()(data, new_data)

data = torch.zeros(3, 4)
new_data = torch.arange(4).to(torch.float32)
torch.onnx.export(LHSIndexing(), (data, new_data), 'inplace_assign.onnx', opset_version=11)

# onnxruntime
import onnxruntime
sess = onnxruntime.InferenceSession('inplace_assign.onnx')
out_ort = sess.run(None, {
    sess.get_inputs()[0].name: torch.zeros(3, 4).numpy(),
    sess.get_inputs()[1].name: new_data.numpy(),
})

assert torch.all(torch.eq(out, torch.tensor(out_ort)))

## Below is the list of supported patterns for LHS indexing.
```
# Scalar indices
data[0, 1] = new_data

# Slice indices
data[:3] = new_data

# Tensor indices
# If more than one tensor are used as indices, only consecutive 1-d tensor indices are supported.
data[torch.tensor([[1, 2], [2, 3]])] = new_data
data[torch.tensor([2, 3]), torch.tensor([1, 2])] = new_data

# Ellipsis
# Not supported to export in script modules
# i.e. torch.onnx.export(torch.jit.script(model)) will fail if model contains this pattern.
# Export is supported under tracing
# i.e. torch.onnx.export(model)
data[...] = new_data

# The combination of above
data[2, ..., torch.tensor([2, 1, 3]), 2:4] += update

# Boolean mask
data[data != 1] = new_data
```